# Step 0: Import python libraries

In [16]:
import requests
import matplotlib.pyplot as plt
import numpy as np
import os
import psutil

from astropy.io import fits


# Step 1: Search for and store BepiColombo data products using the search API

In [25]:
response = requests.get('https://pds.nasa.gov/api/search/1/products?q=((pds:Science_Facets.pds:wavelength_range eq "Visible") '
                        'and (pds:Investigation_Area.pds:name eq "BepiColombo") '
                        'and (pds:Header.pds:parsing_standard_id eq "FITS 3.0")'
                        'and (pds:Time_Coordinates.pds:start_date_time gt "2022-04-30T00:00:00Z")'
                        '  )&start=0&limit=3000')

resp_data = response.json()['data']

KeyboardInterrupt: 

### Store image URLS in a list format to load for visualization

In [15]:
urls = [prod['properties']['ops:Data_File_Info.ops:file_ref'][0] for prod in resp_data]
urls

[]

# Step 2: Grid Visualization of Image Data
Specify the page you would like to see with the page parameter below. Each page displays 48 image products. The current time query is viewing BC's mercury flyby on the 23rd of June 2022.

In [ ]:
#EDIT THIS VARIABLE TO CHANGE PAGES
PAGE = 1
#EDIT THIS VARIABLE TO CHANGE SUBSAMPLING
SUBSAMPLING_RATE = 7 #Indicates how many pixels to skip when plotting

fig, axs = plt.subplots(12,4 , figsize=(8, 24))
NUM_COLUMNS = 4
NUM_ROWS = 12
size = NUM_COLUMNS*NUM_ROWS

for i, image_path in enumerate(urls[size*(PAGE-1):size*PAGE]):
    response = requests.get(image_path[0])
    with fits.open(image_path[0]) as hdul:
        image_data = hdul[1].data
    row_col = divmod(16,4)
    axs[row_col[0], row_col[1]].imshow(image_data[0::SUBSAMPLING_RATE,0::SUBSAMPLING_RATE], cmap='gray')
    #Split the lid by ':' and select the last part of the lid for each image title
    axs[row_col[0], row_col[1]].set_title(resp_data[i]['properties']['lid'][0].split(':')[5], fontsize = 4)
    #Free up memory
    del image_data

for ax in axs.flat:
    ax.label_outer()

plt.suptitle('urn:esa:psa:bc_mtm_mcam:data_raw:', fontsize=16,y=1.01)

plt.tight_layout()
plt.show()
